In [454]:
from io import StringIO
from docx.shared import Cm
from docxtpl import DocxTemplate, InlineImage
import pandas as pd
from math import log10


In [455]:
def qb_dataframe(table_id):
    # Get fields
    url = "https://api.quickbase.com/v1/fields?tableId=" + table_id
    payload = {}
    headers = {
      'QB-Realm-Hostname': 'soteriarf',
      'Authorization': 'QB-USER-TOKEN b4pp4v_paqb_eyxwg9bec3v8pb63hjwjb9t36ii',
      'Content': 'application.json'
    }
    response = requests.request("GET", url, headers=headers, data = payload)
    df = pd.json_normalize(response.json())
    df = df[["label", "id", "fieldType", "mode"]]
    df = df.sort_values('id').reset_index(drop=True)
    df = df[df['fieldType'] != 'user']
    df = df[df['fieldType'] != 'file']
    label_list = list(df['label'])
    id_list = list(df['id'])
    #print(label_list, id_list)
    
    # Query the table
    headers = {
      'QB-Realm-Hostname': 'soteriarf.quickbase.com',
      'Authorization': 'QB-USER-TOKEN b4pp4v_paqb_eyxwg9bec3v8pb63hjwjb9t36ii',
      'Content-Type': 'application/json',
    }
    body = {
        "from": table_id,
        "select": id_list,
        "options": {
            "skip": 0,
            "top": 10000,
            #"compareWithAppLocalTime": false
        }} #, "where": "{6.EQ.'" + site_id + "'}"}
    r = requests.post(
        'https://api.quickbase.com/v1/records/query', 
        headers = headers, 
        json = body
    )
    df = pd.json_normalize(r.json()['data'])
    df.columns = df.columns.str.replace(".value", "")
    
    # Rename the dataframe with the correct fields names
    id_list = [str(i) for i in id_list] 
    for col in df.columns:
        df.rename(columns={col:label_list[id_list.index((col))]}, inplace=True)
    return df

In [456]:
# Function to convert from mW to dBm
def mW2dBm(mW):
    return 10.*log10(mW)

# Function to convert from dBm to mW
def dBm2mW(dBm):
    return 10**((dBm)/10.)

In [457]:
def get_context(row_contents):
    """ You can generate your context separately since you may deal with a lot 
        of documents. You can carry out computations, etc in here and make the
        context look like the sample below.
    """
    return {
        'site_id': "SF14136A",
        'date': '30 Mar',
        'due_date': '30 Apr',
        'name': 'Jane Doe',
        'address': '100 South Mathilda Avenue, Sunnyvale, California 94086',
        'subtotal': 335,
        'tax_amt': 10,
        'total': 345,
        'amt_paid': 100,
        'amt_due': 245,
        'row_contents': row_contents
        # [
        #     {
        #         'sector': 'A',
        #         'technology': 'LTE',
        #         'freq': 2100,
        #         'manufacturer': "Ericsson",
        #         'model': "AIR32",
        #         'azimuth': 0,
        #         'agl': 96,
        #         'power': 40,
        #         'gain': 14.1,
        #         'erp': 2000
        #     }, {
        #         'sector': 'B',
        #         'technology': 'N',
        #         'freq': 2500,
        #         'manufacturer': "Commscope",
        #         'model': "AIR6449",
        #         'azimuth': 120,
        #         'agl': 96,
        #         'power': 80,
        #         'gain': 14.1,
        #         'erp': 2000
        #     }, {
        #         'sector': 'C',
        #         'technology': 'GSM',
        #         'freq': 1900,
        #         'manufacturer': "RFS",
        #         'model': "AIR21",
        #         'azimuth': 240,
        #         'agl': 96,
        #         'power': 120,
        #         'gain': 14.1,
        #         'erp': 2000            }
        # ]
    }
    #Antenna  ManufacturerAntenna ModelAzimuth (°TN)AGL (FT)Antenna Input Power (W)Antenna Gain (dBd)Power Output (ERP), W

In [458]:
# Load Transmitters Table
df = qb_dataframe('bqzaae4x9')

# Query on the site
df = df[df['Antenna Location - SITE_ID'] == site_id]
df.head()

,Date Created,Input Power (W),Date Modified,Pattern Code,Pattern Code - Aperture (Feet),Pattern Code - Gain (dBd),Pattern Code - Horizontal BW,Pattern Code - Vertical BW,Record ID#,Pattern Code - Manufacturer,...,Antenna Location - SITE_ID - Rooftop AGL,Antenna Location - SITE_ID - Roofview AGL,Antenna Location - ANTENNA_ID,Antenna Location - SECTOR,Radio Power (W),Number of Radios,Loss (dB),Antenna Location - RAD_CENTER_FT,Technology,Frequency MHz
0,2020-12-09T00:02:02Z,280.0,2020-12-15T22:50:39Z,AIR6449_2500,2.759183,22.65,60.0,7.5,322,Ericsson,...,92.0,92.0,SF14136A_TMO_A_1,A,280.0,1.0,NaN,96,LTE,2500.0
1,2020-12-09T00:08:46Z,80.0,2020-12-14T17:45:10Z,AIR32_2100,6.466667,15.35,59.0,5.2,326,Ericsson,...,92.0,92.0,SF14136A_TMO_A_2,A,40.0,2.0,NaN,96,,2100.0
2,2020-12-09T00:10:24Z,160.0,2020-12-14T17:45:10Z,AIR32_1900,6.466667,15.35,60.0,5.6,329,Ericsson,...,92.0,92.0,SF14136A_TMO_A_2,A,40.0,4.0,NaN,96,,1900.0
3,2020-12-09T00:12:19Z,80.0,2020-12-14T17:45:10Z,APXVAARR18_43-U-NA20_700,6.000000,12.65,63.0,13.0,332,RFS,...,92.0,92.0,SF14136A_TMO_A_3,A,40.0,2.0,NaN,96,,700.0
4,2020-12-09T00:12:35Z,160.0,2020-12-14T17:45:10Z,APXVAARR18_43-U-NA20_600,6.000000,12.25,66.0,14.0,333,RFS,...,92.0,92.0,SF14136A_TMO_A_3,A,40.0,4.0,NaN,96,,600.0


In [459]:
list(df)

['Date Created',
 'Input Power (W)',
 'Date Modified',
 'Pattern Code',
 'Pattern Code - Aperture (Feet)',
 'Pattern Code - Gain (dBd)',
 'Pattern Code - Horizontal BW',
 'Pattern Code - Vertical BW',
 'Record ID#',
 'Pattern Code - Manufacturer',
 'z_roofview',
 'sort_key',
 'Related Antenna Location',
 'Antenna Location - SITE_ID',
 'Antenna Location - ANTENNA_MODEL',
 'Antenna Location - AZIMUTH',
 'Antenna Location - SITE_ID - Scale',
 'Antenna Location - x_roofview',
 'Antenna Location - y_roofview',
 'Antenna Location - SITE_ID - Rooftop AGL',
 'Antenna Location - SITE_ID - Roofview AGL',
 'Antenna Location - ANTENNA_ID',
 'Antenna Location - SECTOR',
 'Radio Power (W)',
 'Number of Radios',
 'Loss (dB)',
 'Antenna Location - RAD_CENTER_FT',
 'Technology',
 'Frequency MHz']

In [460]:
df = df.rename(columns={
    'Antenna Location - SECTOR': 'sector',
    'Technology': 'technology',
    'Frequency MHz': 'freq',
    'Pattern Code - Manufacturer': 'manufacturer',
    'Antenna Location - ANTENNA_MODEL': 'model',
    'Antenna Location - AZIMUTH': 'azimuth',
    'Antenna Location - RAD_CENTER_FT': 'agl',
    'Input Power (W)': 'power',
    'Pattern Code - Gain (dBd)': 'gain'
    })
df.head()

,Date Created,power,Date Modified,Pattern Code,Pattern Code - Aperture (Feet),gain,Pattern Code - Horizontal BW,Pattern Code - Vertical BW,Record ID#,manufacturer,...,Antenna Location - SITE_ID - Rooftop AGL,Antenna Location - SITE_ID - Roofview AGL,Antenna Location - ANTENNA_ID,sector,Radio Power (W),Number of Radios,Loss (dB),agl,technology,freq
0,2020-12-09T00:02:02Z,280.0,2020-12-15T22:50:39Z,AIR6449_2500,2.759183,22.65,60.0,7.5,322,Ericsson,...,92.0,92.0,SF14136A_TMO_A_1,A,280.0,1.0,NaN,96,LTE,2500.0
1,2020-12-09T00:08:46Z,80.0,2020-12-14T17:45:10Z,AIR32_2100,6.466667,15.35,59.0,5.2,326,Ericsson,...,92.0,92.0,SF14136A_TMO_A_2,A,40.0,2.0,NaN,96,,2100.0
2,2020-12-09T00:10:24Z,160.0,2020-12-14T17:45:10Z,AIR32_1900,6.466667,15.35,60.0,5.6,329,Ericsson,...,92.0,92.0,SF14136A_TMO_A_2,A,40.0,4.0,NaN,96,,1900.0
3,2020-12-09T00:12:19Z,80.0,2020-12-14T17:45:10Z,APXVAARR18_43-U-NA20_700,6.000000,12.65,63.0,13.0,332,RFS,...,92.0,92.0,SF14136A_TMO_A_3,A,40.0,2.0,NaN,96,,700.0
4,2020-12-09T00:12:35Z,160.0,2020-12-14T17:45:10Z,APXVAARR18_43-U-NA20_600,6.000000,12.25,66.0,14.0,333,RFS,...,92.0,92.0,SF14136A_TMO_A_3,A,40.0,4.0,NaN,96,,600.0


In [461]:
df = df[['sector', 'technology', 'freq', 'manufacturer', 'model', 'azimuth', 'agl', 'power', 'gain']]
df.head()

,sector,technology,freq,manufacturer,model,azimuth,agl,power,gain
0,A,LTE,2500.0,Ericsson,AIR6449,10.0,96,280.0,22.65
1,A,,2100.0,Ericsson,AIR32,10.0,96,80.0,15.35
2,A,,1900.0,Ericsson,AIR32,10.0,96,160.0,15.35
3,A,,700.0,RFS,APXVAARR18_43-U-NA20,10.0,96,80.0,12.65
4,A,,600.0,RFS,APXVAARR18_43-U-NA20,10.0,96,160.0,12.25


In [462]:
df['erp_dbm'] = ''
df['erp'] = ''
for x in range(len(df)):
    df['erp_dbm'].iloc[x] = round(mW2dBm(df['power'].iloc[x]*1000) + df['gain'].iloc[x], 1)
    df['erp'].iloc[x] = int(dBm2mW(df['erp_dbm'].iloc[x]) / 1000)
df = df.drop('erp_dbm', 1)
df.head()

,sector,technology,freq,manufacturer,model,azimuth,agl,power,gain,erp
0,A,LTE,2500.0,Ericsson,AIR6449,10.0,96,280.0,22.65,51286
1,A,,2100.0,Ericsson,AIR32,10.0,96,80.0,15.35,2754
2,A,,1900.0,Ericsson,AIR32,10.0,96,160.0,15.35,5495
3,A,,700.0,RFS,APXVAARR18_43-U-NA20,10.0,96,80.0,12.65,1479
4,A,,600.0,RFS,APXVAARR18_43-U-NA20,10.0,96,160.0,12.25,2691


In [463]:
df.iloc[0].to_json(r'json_out.json')
# {"sector":"A","technology":"LTE","freq":2500.0,"manufacturer":"Ericsson","model":"AIR6449","azimuth":10.0,"agl":"96","power":280.0,"gain":22.65,"erp_dbm":77.1,"erp":51286}

In [464]:
row_contents = []
for x in range(len(df)):
    print(df.iloc[x].to_json())
    row_contents.append(eval(df.iloc[x].to_json()))
row_contents

{"sector":"A","technology":"LTE","freq":2500.0,"manufacturer":"Ericsson","model":"AIR6449","azimuth":10.0,"agl":"96","power":280.0,"gain":22.65,"erp":51286}
{"sector":"A","technology":"","freq":2100.0,"manufacturer":"Ericsson","model":"AIR32","azimuth":10.0,"agl":"96","power":80.0,"gain":15.35,"erp":2754}
{"sector":"A","technology":"","freq":1900.0,"manufacturer":"Ericsson","model":"AIR32","azimuth":10.0,"agl":"96","power":160.0,"gain":15.35,"erp":5495}
{"sector":"A","technology":"","freq":700.0,"manufacturer":"RFS","model":"APXVAARR18_43-U-NA20","azimuth":10.0,"agl":"96","power":80.0,"gain":12.65,"erp":1479}
{"sector":"A","technology":"","freq":600.0,"manufacturer":"RFS","model":"APXVAARR18_43-U-NA20","azimuth":10.0,"agl":"96","power":160.0,"gain":12.25,"erp":2691}
{"sector":"A","technology":"","freq":1900.0,"manufacturer":"RFS","model":"APXVAARR18_43-U-NA20","azimuth":10.0,"agl":"96","power":160.0,"gain":14.85,"erp":4897}
{"sector":"B","technology":"","freq":2500.0,"manufacturer":"Er

['"{\\"sector\\":\\"A\\",\\"technology\\":\\"LTE\\",\\"freq\\":2500.0,\\"manufacturer\\":\\"Ericsson\\",\\"model\\":\\"AIR6449\\",\\"azimuth\\":10.0,\\"agl\\":\\"96\\",\\"power\\":280.0,\\"gain\\":22.65,\\"erp\\":51286}"',
 '"{\\"sector\\":\\"A\\",\\"technology\\":\\"\\",\\"freq\\":2100.0,\\"manufacturer\\":\\"Ericsson\\",\\"model\\":\\"AIR32\\",\\"azimuth\\":10.0,\\"agl\\":\\"96\\",\\"power\\":80.0,\\"gain\\":15.35,\\"erp\\":2754}"',
 '"{\\"sector\\":\\"A\\",\\"technology\\":\\"\\",\\"freq\\":1900.0,\\"manufacturer\\":\\"Ericsson\\",\\"model\\":\\"AIR32\\",\\"azimuth\\":10.0,\\"agl\\":\\"96\\",\\"power\\":160.0,\\"gain\\":15.35,\\"erp\\":5495}"',
 '"{\\"sector\\":\\"A\\",\\"technology\\":\\"\\",\\"freq\\":700.0,\\"manufacturer\\":\\"RFS\\",\\"model\\":\\"APXVAARR18_43-U-NA20\\",\\"azimuth\\":10.0,\\"agl\\":\\"96\\",\\"power\\":80.0,\\"gain\\":12.65,\\"erp\\":1479}"',
 '"{\\"sector\\":\\"A\\",\\"technology\\":\\"\\",\\"freq\\":600.0,\\"manufacturer\\":\\"RFS\\",\\"model\\":\\"APXVAARR1

In [465]:
get_context(row_contents)

{'site_id': 'SF14136A',
 'date': '30 Mar',
 'due_date': '30 Apr',
 'name': 'Jane Doe',
 'address': '100 South Mathilda Avenue, Sunnyvale, California 94086',
 'subtotal': 335,
 'tax_amt': 10,
 'total': 345,
 'amt_paid': 100,
 'amt_due': 245,
 'row_contents': ['"{\\"sector\\":\\"A\\",\\"technology\\":\\"LTE\\",\\"freq\\":2500.0,\\"manufacturer\\":\\"Ericsson\\",\\"model\\":\\"AIR6449\\",\\"azimuth\\":10.0,\\"agl\\":\\"96\\",\\"power\\":280.0,\\"gain\\":22.65,\\"erp\\":51286}"',
  '"{\\"sector\\":\\"A\\",\\"technology\\":\\"\\",\\"freq\\":2100.0,\\"manufacturer\\":\\"Ericsson\\",\\"model\\":\\"AIR32\\",\\"azimuth\\":10.0,\\"agl\\":\\"96\\",\\"power\\":80.0,\\"gain\\":15.35,\\"erp\\":2754}"',
  '"{\\"sector\\":\\"A\\",\\"technology\\":\\"\\",\\"freq\\":1900.0,\\"manufacturer\\":\\"Ericsson\\",\\"model\\":\\"AIR32\\",\\"azimuth\\":10.0,\\"agl\\":\\"96\\",\\"power\\":160.0,\\"gain\\":15.35,\\"erp\\":5495}"',
  '"{\\"sector\\":\\"A\\",\\"technology\\":\\"\\",\\"freq\\":700.0,\\"manufacturer\\

In [466]:
doc = DocxTemplate('AntennaInventoryTpl.docx')
context = get_context(row_contents)
doc.render(context)
doc.save('test.docx')